In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

### PLAN

1. Pobranie danych
2. Przejrzeć co mamy
    - wielkosc data setu
    - jakie kolumny
    - jakiego typu dane
    - podejrzec probke
    - jesli dane sa we wlasciwym formacie: proste statystyki
3. Konwersja danych na odpowiedni format
4. Co z brakujacymi wartosciami?

#### Zaimportuj i przejrzyj dane

In [2]:
dane = pd.read_csv('data/DataWorkshop.csv')

In [3]:
print('Rozmiar danych: ', dane.shape)

Rozmiar danych:  (229929, 30)


In [4]:
print('Kolumny w datasecie: ')
dane.columns

Kolumny w datasecie: 


Index(['Data zdarzenia', 'Godzina', 'GPS x', 'GPS y', 'Miejscowość', 'Ulica',
       'Numer domu', 'Numer drogi', 'KM HM (Pikietaż)', 'Obszar',
       'Skrzyżowanie', 'Geometria drogi', 'Charakterystyka miejsca zdarzenia',
       'Oświetlenie', 'Warunki atmosferyczne', 'Rodzaj zdarzenia', 'Wina',
       'Rodzaj drogi', 'Nawierzchnia drogi', 'Stan nawierzchni',
       'Sygnalizacja świetlna', 'Oznakowanie poziome',
       'Liczba pasów w zdarzeniu', 'Zachowanie kierującego',
       'Zachowanie pieszego', 'Inne przyczyny', 'Liczba rannych w zdarzeniu',
       'Liczba zabitych w zdarzeniu', 'Liczba kolizji', 'Liczba wypadków'],
      dtype='object')

In [5]:
print('Typy danych: ')
dane.dtypes

Typy danych: 


Data zdarzenia                        object
Godzina                               object
GPS x                                 object
GPS y                                 object
Miejscowość                           object
Ulica                                 object
Numer domu                            object
Numer drogi                           object
KM HM (Pikietaż)                      object
Obszar                                object
Skrzyżowanie                          object
Geometria drogi                       object
Charakterystyka miejsca zdarzenia     object
Oświetlenie                           object
Warunki atmosferyczne                 object
Rodzaj zdarzenia                      object
Wina                                  object
Rodzaj drogi                          object
Nawierzchnia drogi                    object
Stan nawierzchni                      object
Sygnalizacja świetlna                 object
Oznakowanie poziome                   object
Liczba pas

In [6]:
print('Brakujace wartosci: ')

missing = dane.isnull().sum()
print(missing)

Brakujace wartosci: 
Data zdarzenia                            0
Godzina                                   0
GPS x                                     0
GPS y                                     0
Miejscowość                               0
Ulica                                 20728
Numer domu                            93884
Numer drogi                               0
KM HM (Pikietaż)                     194888
Obszar                                    1
Skrzyżowanie                         166779
Geometria drogi                       66008
Charakterystyka miejsca zdarzenia         2
Oświetlenie                               0
Warunki atmosferyczne                     1
Rodzaj zdarzenia                          0
Wina                                      0
Rodzaj drogi                              1
Nawierzchnia drogi                        1
Stan nawierzchni                          1
Sygnalizacja świetlna                     0
Oznakowanie poziome                       2
Liczba pasó

In [7]:
print('Brakujace wartosci jako procent zbioru, %: ')
index=[]
lst=[]
for item in missing.iteritems():
    index.append(item[0])
    lst.append(round((item[1]/dane.shape[0])*100,4))
missing_percentage=pd.Series(lst, index=index)
print(missing_percentage)

Brakujace wartosci jako procent zbioru, %: 
Data zdarzenia                        0.0000
Godzina                               0.0000
GPS x                                 0.0000
GPS y                                 0.0000
Miejscowość                           0.0000
Ulica                                 9.0150
Numer domu                           40.8317
Numer drogi                           0.0000
KM HM (Pikietaż)                     84.7601
Obszar                                0.0004
Skrzyżowanie                         72.5350
Geometria drogi                      28.7080
Charakterystyka miejsca zdarzenia     0.0009
Oświetlenie                           0.0000
Warunki atmosferyczne                 0.0004
Rodzaj zdarzenia                      0.0000
Wina                                  0.0000
Rodzaj drogi                          0.0004
Nawierzchnia drogi                    0.0004
Stan nawierzchni                      0.0004
Sygnalizacja świetlna                 0.0000
Oznakowanie

In [8]:
# Probka danych
dane.sample(50)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
173098,2019-01-27,17:00:00,19*07'199,50*48'408,CZĘSTOCHOWA,MARSZAŁKA JÓZEFA PIŁSUDSKIEGO,2/6,P6635S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,NaN,NaN,Inne,0,0,1,0
44027,2016-11-06,16:30:00,18*49'258,50*26'404,TARNOWSKIE GÓRY,WŁOSKA,NaN,G270 331S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0
54028,2017-01-07,11:00:00,19*00'288,50*15'446,KATOWICE,GLIWICKA,NaN,P6431S,NaN,Obszar zabudowany,...,Brak,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0
212486,2019-09-27,00:35:00,19*26'044,50*28'569,ZAWIERCIE,IGNACEGO PADEREWSKIEGO,23,78,"120,6",Obszar zabudowany,...,Brak,Jest,2.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0
193476,2019-05-31,16:13:00,18*50'208,50*08'324,ŁAZISKA GÓRNE,GÓRNA,16,G450058S,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0
132421,2018-05-23,12:30:00,19*04'190,49*53'198,JANOWICE,PODLESIE,18,GMINNA,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0
8063,2016-03-03,13:45:00,18*54'491,50*17'088,ŚWIĘTOCHŁOWICE,METALOWCÓW,1,P9809S,NaN,Obszar zabudowany,...,Brak,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
143977,2018-08-08,08:41:00,19*15'245,50*20'054,DĄBROWA GÓRNICZA,BUDOWLANYCH,7,G200 096S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0
192434,2019-05-25,11:00:00,18*54'121,50*15'125,RUDA ŚLĄSKA,OŚWIĘCIMSKA,NaN,P8402S,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
11105,2016-03-27,22:20:00,19*23'589,50*27'275,KUŹNICA MASŁOŃSKA,TOPOLOWA,5B,G570604S,NaN,Obszar niezabudowany,...,Brak,Nie ma,NaN,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0


### Konwersja typu danych

#### Data i czas

In [9]:
# Utworzenie kopii zbioru danych. Dalsza konwersja odbywac sie bedzie na kopii danych
df=dane.copy()

In [10]:
# Data zdarzenia i Godzina scalone do jednej kolumny
print('kolumna DATA ZDARZENIA: ', df['Data zdarzenia'].dtypes)
print('kolumna GODZINA: ', df['Data zdarzenia'].dtypes)
df['data_full'] = df['Data zdarzenia'] + ' ' + df['Godzina']
df.sample(10)

kolumna DATA ZDARZENIA:  object
kolumna GODZINA:  object


,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
200085,2019-07-10,19:15:00,19*13'392,49*40'209,ŻYWIEC,KOPERNIKA,NaN,W945,"15,3",Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-07-10 19:15:00
173099,2019-01-28,08:25:00,18*33'078,50*05'573,RYBNIK,PIASTA,35,G180296S,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-01-28 08:25:00
170042,2019-01-09,10:00:00,18*58'219,50*06'180,TYCHY,BIELSKA,135,P8004S,NaN,Obszar zabudowany,...,Nie ma,2.0,Inne przyczyny,NaN,NaN,0,0,1,0,2019-01-09 10:00:00
211216,2019-09-19,14:30:00,19*13'415,50*10'196,MYSŁOWICE,DŁUGA,166,P8800S,NaN,Obszar zabudowany,...,Nie ma,2.0,NaN,NaN,Stan jezdni,0,0,1,0,2019-09-19 14:30:00
150744,2018-09-20,09:58:00,19*25'422,50*29'038,ZAWIERCIE,IGNACEGO PADEREWSKIEGO,5,78,120,Obszar zabudowany,...,Jest,3.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2018-09-20 09:58:00
161597,2018-11-22,13:39:00,18*56'139,50*16'017,CHORZÓW,STEFANA BATOREGO,9,P9405S,NaN,Obszar zabudowany,...,Jest,3.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-11-22 13:39:00
155162,2018-10-15,11:30:00,19*15'162,50*57'180,BOGUSŁAWICE,GRABOWSKA,NaN,G694001S,NaN,Obszar zabudowany,...,Jest,2.0,Nieprawidłowe wymijanie,NaN,NaN,0,0,1,0,2018-10-15 11:30:00
200509,2019-07-13,09:25:00,19*02'195,50*44'288,WYGODA,NaN,28,W908,NaN,Obszar zabudowany,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2019-07-13 09:25:00
130539,2018-05-12,08:55:00,19*02'078,49*51'026,BIELSKO-BIAŁA,KATOWICKA,NaN,P7405S,NaN,Obszar zabudowany,...,Jest,2.0,Nieprzestrzeganie znaków i innych sygnałów,NaN,NaN,0,0,1,0,2018-05-12 08:55:00
81107,2017-07-08,18:20:00,19*04'198,50*13'087,KATOWICE,ADAMA,3,G100034S,NaN,Obszar zabudowany,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2017-07-08 18:20:00


In [11]:
# Data zdarzenia i Godzina do typu datetime
df['data_full'] = pd.to_datetime(df['data_full'], format='%Y-%m-%d %H:%M:%S')
df.sample(10)
# TO-DO: Jaki to dzien tygodnia? Jaki to dzien miesiaca? To dzien czy noc? Jaka faza ksiezyca itd..

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
81113,2017-07-08,20:40:00,19*07'447,49*36'432,WĘGIERSKA GÓRKA,ZACISZE,13,G596052S,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2017-07-08 20:40:00
193972,2019-06-03,15:10:00,19*01'136,49*48'209,BIELSKO-BIAŁA,GOLESZOWSKA,16,I2,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2019-06-03 15:10:00
47470,2016-11-28,20:10:00,18*57'050,50*16'057,CHORZÓW,INWALIDZKA,NaN,G210084S,NaN,Obszar zabudowany,...,Jest,2.0,Inne przyczyny,NaN,NaN,0,0,1,0,2016-11-28 20:10:00
129624,2018-05-07,10:10:00,19*00'099,50*14'161,KATOWICE,KOŚCIUSZKI,NaN,P6458S,NaN,Obszar zabudowany,...,Jest,4.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2018-05-07 10:10:00
219192,2019-10-31,10:55:00,19*04'067,50*13'362,KATOWICE,SZOPIENICKA,NaN,P6534S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa pieszemu na przejś...,NaN,NaN,0,1,0,1,2019-10-31 10:55:00
111791,2018-01-08,11:23:00,19*06'428,50*48'401,CZĘSTOCHOWA,SZYMANOWSKIEGO,3,G110860S,NaN,Obszar zabudowany,...,Nie ma,2.0,NaN,NaN,Stan jezdni,0,0,1,0,2018-01-08 11:23:00
54848,2017-01-12,07:00:00,18*50'413,50*17'559,ZABRZE,DROGOWA TRASA ŚREDNICOWA,NIEZNANE,W902,NaN,Obszar niezabudowany,...,Jest,NaN,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2017-01-12 07:00:00
61880,2017-02-15,17:30:00,18*11'463,50*02'358,RACIBÓRZ,RACIBORSKA,NaN,45,16,Obszar niezabudowany,...,Jest,NaN,Nieprawidłowe wyprzedzanie,NaN,NaN,0,0,1,0,2017-02-15 17:30:00
145401,2018-08-17,14:28:00,19*08'580,50*35'536,KOZIEGŁOWY,NaN,NIEZNANE,1,"500,6",Obszar niezabudowany,...,Jest,4.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-08-17 14:28:00
176714,2019-02-17,17:40:00,19*02'558,49*55'468,KANIÓW,WINCENTEGO WITOSA,38,P4403S,NaN,Obszar zabudowany,...,Jest,2.0,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0,2019-02-17 17:40:00


#### Wspolrzedne geograficzne

Domyslny zapis wspolrzednych to D\*M'S([D]egree, [M]inute, [S]cond) np. 18\*53'289. Jednak, gdy pierwsza cyfra sekund jest zero, czasem wystepuje zapis D\*M''S np. 18\*53''11. Wystepowaly takze bledne zapisy takie jak 4\*\*56'569 i 50\*09'\*01.

In [12]:
def dmsToDecimal(coordinate_as_string):
    try:
        # Zapis domyslny to D*M'S([D]egree, [M]inute, [S]cond) zeby sprawnie podzielic kazda wspolrzedna na liste trzyelementowa
        # zamieniam ' na *, a nastepnie dziele orzymany rezultat znakami *. W trakcie konwersji danych napotkalam rozne nieprawidlowe
        # zapisy opisane w komorce powyzej. Wszystkie wystepujace nietypowe znaki takie jak **, '* i '' zamieniam na * i dziele znakiem *
        degree, minute, second = str(coordinate_as_string).replace("**", "*").replace("'*", "*").replace("''", "*").replace("'", '*').split("*")
    except:
        # Jesli nie dzieli na liste trzyelementowa to najprawdopodobniej trzeci element jest pusty(lista dwuelementowa).
        # Wtedy pierwszy element to stopien, drugi minuta. Sekundy zas sa rowne 0.
        degree, minute = str(coordinate_as_string).replace("**", "*").replace("'*", "*").replace("''", "*").replace("'", '*').split("*")
        second=0
    # W niektorych wypadkach udaje sie podzielic wsadowego stringa tak aby uzyskac ostatnia czesc jako pusty string.
    #Ponizej przeksztalcam pustego stringa w wartosc 0
    if second == "" or second == " ":
        second=0
    # Wlasciwe przeksztalcenie wspolrzednej z wartosci katowych na wartosc liczbowa
    coordinate_as_float = float(degree) + float(minute)/60 + float(second)/(60*60)
    return coordinate_as_float
# Przygotowana wyzej funkcje wywoluje na calych kolumnach dancych
df['GPS x'] = df['GPS x'].apply(lambda x: dmsToDecimal(x))
df['GPS y'] = df['GPS y'].apply(lambda x: dmsToDecimal(x))
df.sample(10)
# TO-DO Wyszukac i wykorzystac biblioteke okreslajaca wspolrzedne GPS na podstawie adresu

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
139899,2018-07-12,13:20:00,19.061111,50.151111,TYCHY,JÓZEFA PIŁSUDSKIEGO,36,P8012S,NaN,Obszar zabudowany,...,Jest,4.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-07-12 13:20:00
169718,2019-01-04,20:15:00,18.671667,50.378889,GLIWICE,KOZIELSKA,608,P7200S,NaN,Obszar zabudowany,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2019-01-04 20:15:00
114477,2018-01-25,09:00:00,18.773611,50.251111,KNURÓW,DWORCOWA,15,W921,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-01-25 09:00:00
56219,2017-01-22,10:24:00,19.315278,50.244444,MYSŁOWICE,NaN,NaN,S1,"548,5",Obszar niezabudowany,...,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2017-01-22 10:24:00
100896,2017-11-05,01:16:00,19.217778,50.858333,CZĘSTOCHOWA,MOŚCICKIEGO,18,G110550S,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2017-11-05 01:16:00
223102,2019-11-22,10:15:00,18.678611,50.121944,RYBNIK,STEFANA ŻEROMSKIEGO,12,G180526S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe wymijanie; Nieprawidłowe wymijanie,NaN,NaN,0,0,1,0,2019-11-22 10:15:00
126024,2018-04-12,17:30:00,19.332500,49.739167,ŻYWIEC,SPORYSKA,NaN,P1478S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-04-12 17:30:00
106105,2017-12-04,17:25:00,19.096667,50.271111,KATOWICE,BRYNOWSKA,NaN,P6453S,NaN,Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2017-12-04 17:25:00
228420,2019-12-23,16:30:00,18.992500,50.205278,TYCHY,BIBLIOTECZNA,19,G190220S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2019-12-23 16:30:00
203644,2019-08-03,15:20:00,18.952500,50.181111,MIKOŁÓW,KROKUSÓW,10,I2,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2019-08-03 15:20:00


#### Miejsce zdarzenia



In [13]:
# Zmiana nazwy kolumn na nazwy bez spacji i polskich znakow
# df.rename(columns={'Miejscowość': 'Miejscowosc', 'Numer domu': "NrDomu", "Numer drogi": "NrDrogi"}, inplace=True)
df.columns

Index(['Data zdarzenia', 'Godzina', 'GPS x', 'GPS y', 'Miejscowość', 'Ulica',
       'Numer domu', 'Numer drogi', 'KM HM (Pikietaż)', 'Obszar',
       'Skrzyżowanie', 'Geometria drogi', 'Charakterystyka miejsca zdarzenia',
       'Oświetlenie', 'Warunki atmosferyczne', 'Rodzaj zdarzenia', 'Wina',
       'Rodzaj drogi', 'Nawierzchnia drogi', 'Stan nawierzchni',
       'Sygnalizacja świetlna', 'Oznakowanie poziome',
       'Liczba pasów w zdarzeniu', 'Zachowanie kierującego',
       'Zachowanie pieszego', 'Inne przyczyny', 'Liczba rannych w zdarzeniu',
       'Liczba zabitych w zdarzeniu', 'Liczba kolizji', 'Liczba wypadków',
       'data_full'],
      dtype='object')

In [14]:
# Pikietaz okresla miejsce zdarzenia na drodze poza terenem zabudowanym
#Zapisywany jako xx,yyy gdzie:
#xx – całkowita liczba kilometrów od początku drogi;
#yyy – całkowita liczba metrów liczona od ostatniego pełnego kilometra.
# W zbiorze danych prawdopodobnie jako wartosc w km, tyle ze z uzyciem "," jako separatora dziesietnego
def pikietazToFloat(pikietazRaw):
    if isinstance(pikietazRaw, str):
        pikietazAsFloat = float(pikietazRaw.replace(",", "."))
    else:
        pikietazAsFloat = pikietazRaw
    return pikietazAsFloat
df['KM HM (Pikietaż)']=df['KM HM (Pikietaż)'].apply(lambda x: pikietazToFloat(x))
# df.rename(columns={'KM HM (Pikietaż)': 'KM_HM_Pikietaz'})
df.sample(10)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
210121,2019-09-12,17:15:00,19.259444,50.372500,SOSNOWIEC,STANISŁAWA MAŁACHOWSKIEGO,5,G120033S,NaN,Obszar zabudowany,...,Nie ma,1.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2019-09-12 17:15:00
201411,2019-07-19,12:01:00,19.281111,50.791944,PORAJ,WESOŁA,23,G625049S,NaN,Obszar niezabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2019-07-19 12:01:00
184513,2019-04-08,12:40:00,19.224722,50.936944,CZĘSTOCHOWA,DĄBROWSKIEGO,7,P6610S,NaN,Obszar zabudowany,...,Jest,2.0,Inne przyczyny,NaN,NaN,0,0,1,0,2019-04-08 12:40:00
61590,2017-02-28,05:40:00,19.096944,50.382222,SIEMIANOWICE ŚLĄSKIE,ŚLĄSKA,59,P9612S,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2017-02-28 05:40:00
64135,2017-03-19,18:30:00,19.056389,49.866667,BIELSKO-BIAŁA,PL.RATUSZOWY,3,P7413S,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2017-03-19 18:30:00
221318,2019-11-13,10:15:00,19.100000,50.336944,KATOWICE,TRASA NIKODEMA I JÓZEFA RENCÓW,33,W902,NaN,Obszar niezabudowany,...,Jest,6.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2019-11-13 10:15:00
32952,2016-08-30,16:05:00,19.025278,50.386111,KATOWICE,SOKOLSKA,NaN,P6519S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0,2016-08-30 16:05:00
104950,2017-11-28,17:00:00,19.034167,50.360556,SIEMIANOWICE ŚLĄSKIE,POLSKIEGO CZERWONEGO KRZYŻA,1,P9641S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2017-11-28 17:00:00
24291,2016-06-25,18:25:00,19.328056,50.297500,JAWORZNO,JANA PAWŁA II,NaN,79,NaN,Obszar zabudowany,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2016-06-25 18:25:00
49491,2016-12-10,09:12:00,18.863889,50.143611,SUSZEC,PSZCZYŃSKA,1,W935,NaN,Obszar zabudowany,...,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-12-10 09:12:00


In [15]:
# Obszar: zabudowany/niezabudowany
# Przypisane zostaly wartosci logiczne: Obszar zabudowany-True, Obszar niezabudowany False.
# W razie potrzeby takie wartosci mozna wygodnie przeksztalcic na int o wartosciach odpowiednio 1 i 0
df['Obszar'].value_counts()
df['Obszar'] = df['Obszar'].apply(lambda x: True if x=='Obszar zabudowany' else False)
df.sample(10)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
216572,2019-10-17,14:30:00,19.280000,50.462222,DĄBROWA GÓRNICZA,ŚW. ANTONIEGO,NaN,W796,NaN,True,...,Jest,2.0,NaN,NaN,Stan jezdni,0,0,1,0,2019-10-17 14:30:00
3194,2016-01-28,19:25:00,18.928611,50.347778,RUDA ŚLĄSKA,1 MAJA,NaN,W925,NaN,True,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-01-28 19:25:00
9176,2016-03-05,11:00:00,18.765556,50.334722,GLIWICE,NaN,L04L,A1A,0.5,False,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2016-03-05 11:00:00
80872,2017-07-05,15:55:00,19.037500,50.392500,SIEMIANOWICE ŚLĄSKIE,BYTKOWSKA,19,P9602S,NaN,True,...,Jest,2.0,Nieprawidłowe Zawracanie,NaN,NaN,1,0,0,1,2017-07-05 15:55:00
121797,2018-03-15,08:05:00,18.956944,50.182778,MIKOŁÓW,ŚW.WOJCIECHA,25,G380092S,NaN,True,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2018-03-15 08:05:00
51200,2016-12-20,10:00:00,18.875556,50.185833,ORZESZE,CENTRALNA,NaN,81,22.5,False,...,Jest,4.0,"Zmęczenie, zaśnięcie",NaN,NaN,1,0,0,1,2016-12-20 10:00:00
144953,2018-08-14,12:05:00,19.080000,50.390278,KATOWICE,RONDO GEN.JERZEGO ZIĘTKA,NaN,79,NaN,True,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-08-14 12:05:00
177450,2019-02-22,15:40:00,19.083056,50.347500,KATOWICE,KOŚCIUSZKI,243,81,NaN,True,...,Jest,2.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2019-02-22 15:40:00
124281,2018-03-31,15:00:00,19.049167,50.334167,KATOWICE,KAROLA HOPPEGO,4A,G100130S,NaN,True,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018-03-31 15:00:00
5034,2016-02-11,08:10:00,18.484722,50.070000,RADLIN,RYMERA,55,P5028S,NaN,True,...,Jest,2.0,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-02-11 08:10:00


In [16]:
# Skrzyzowanie
# Zmiana nazwy na niezawierajaca polskich znakow
df['Skrzyżowanie'].unique()
df['Skrzyżowanie'].value_counts()
# Zamiana wartosci tekstowych na int zgodnie z zasada:
# 0 : O ruchu okrężnym; 1 : Równorzędne; 2 : Z drogą z pierwsz.
skrzyzowaniemap = {'O ruchu okrężnym':0, 'Równorzędne':1, 'Z drogą z pierwsz.':2}
df=df.applymap(lambda x: skrzyzowaniemap.get(x) if x in skrzyzowaniemap else x)
# df.rename(columns={'Skrzyżowanie': 'Skrzyzowanie'})

In [17]:
df['Geometria drogi'].value_counts()
# Geometria pozioma: 
# 0: Odcinek prosty; 1: Zakret(luk)

# Geometria pionowa: 
# 0: Spadek; 1: wzniesienie; 2: wierzcholek wzniesienia

Odcinek prosty                      142703
Zakręt, łuk                          16917
Spadek                                1866
Wzniesienie                           1317
Odcinek prosty; Spadek                 430
Odcinek prosty; Wzniesienie            329
Zakręt, łuk; Spadek                    179
Zakręt, łuk; Wzniesienie               108
Wierzchołek wzn.                        50
Odcinek prosty; Wierzchołek wzn.        16
Zakręt, łuk; Wierzchołek wzn.            5
Odcinek prosty; Zakręt, łuk              1
Name: Geometria drogi, dtype: int64

In [18]:
df['Geometria_drogi_poziom'] = df['Geometria drogi'].copy()
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 0 if 'Odcinek prosty' in str(x) else x)
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 1 if 'Zakręt' in str(x) else x)
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 1 if 'łuk' in str(x) else x)
# Co z reszta? Tzn z sytuacja kiedy nie jest zdefiniowane czy droga byla prosta czy byl to zakret?
df['Geometria_drogi_poziom'].value_counts()

0                   143479
1                    17209
Spadek                1866
Wzniesienie           1317
Wierzchołek wzn.        50
Name: Geometria_drogi_poziom, dtype: int64

In [19]:
df['Geometria_drogi_pion'] = df['Geometria drogi'].copy()
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 0 if 'Spadek' in str(x) else x)
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 1 if 'Wzniesienie' in str(x) else x)
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 2 if 'Wierzchołek wzn.' in str(x) else x)
# Co z reszta? Tzn z sytuacja kiedy nie jest zdefiniowane uksztaltowanie terenu?
df['Geometria_drogi_pion'].value_counts()

Odcinek prosty                 142703
Zakręt, łuk                     16917
0                                2475
1                                1754
2                                  71
Odcinek prosty; Zakręt, łuk         1
Name: Geometria_drogi_pion, dtype: int64